In [ ]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-fe0f894b-bbd8-ac71-defd-9dc503fc01d3)


In [ ]:
!pip install --upgrade --quiet bitsandbytes datasets evaluate peft tensorboard transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 133.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 57.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires tensorboard~=2.19.0, but you have tensorboard 2.20.0 which is incompatible.


In [ ]:
import torch
import evaluate
from typing import Any
from transformers import pipeline
from trl import SFTConfig,SFTTrainer
from peft import LoraConfig,get_peft_model
from datasets import load_dataset,ClassLabel
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

In [ ]:
import random
import json
from datasets import concatenate_datasets

# Load WebClick dataset from Hugging Face
dataset = load_dataset("Hcompany/WebClick")

# WebClick only has 'test' split, so create train/val split
dataset = dataset["test"].train_test_split(test_size=0.2, seed=42)

data = {
    "train": dataset["train"],
    "validation": dataset["test"]
}

print(f"Train samples: {len(data['train'])}")
print(f"Validation samples: {len(data['validation'])}")

README.md: 0.00B [00:00, ?B/s]

test-00000-of-00004.parquet:   0%|          | 0.00/92.1M [00:00<?, ?B/s]

test-00001-of-00004.parquet:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

test-00002-of-00004.parquet:   0%|          | 0.00/100M [00:00<?, ?B/s]

test-00003-of-00004.parquet:   0%|          | 0.00/43.1M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1639 [00:00<?, ? examples/s]

Train samples: 1311
Validation samples: 328


In [ ]:
data["train"][0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1096x824>,
 'instruction': 'Click 8.45 heated terrace',
 'bbox': [0.47292712135036497,
  0.6317885315533981,
  0.6852189781021898,
  0.7004702669902912],
 'bucket': 'calendars'}

In [ ]:
# Define question templates for random selection
QUESTION_TEMPLATES = [
    "What action should I take on this page?",
    "Where should I click here?",
    "Which element should I click?",
    "What is the next step?",
    "How do I complete this task?",
    "What should I do next?",
    "Where is the interactive element?",
    "What button or link should I click?",
]


def format_webclick_with_varied_questions(example: dict[str, Any]) -> dict[str, Any]:
    instruction = example["instruction"]  # e.g., "Navigate to the 'Learn' section"
    bbox = example["bbox"]  # [x1, y1, x2, y2] normalized 0-1
    category = example["bucket"]  # e.g., "agentbrowse", "calendars"

    # Randomly select question template
    selected_question = random.choice(QUESTION_TEMPLATES)

    # Create JSON output (consistent format regardless of question)
    x1_int = int(bbox[0] * 100)
    y1_int = int(bbox[1] * 100)
    x2_int = int(bbox[2] * 100)
    y2_int = int(bbox[3] * 100)

    # Create JSON output with integer coordinates
    output_json = {
        "instruction": instruction,
        "bounding_box": {
            "x1": x1_int,
            "y1": y1_int,
            "x2": x2_int,
            "y2": y2_int
        },
        "category": category
    }

    # Create messages for chat template
    example["messages"] = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": " ".join("""You are a web automation assistant.
                    Analyze the webpage screenshot and respond with valid JSON containing 'instruction',
                    'bounding_box' with x1/y1/x2/y2 coordinates, and 'category'.
                    """).splitlines()
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": selected_question},
            ],
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": json.dumps(output_json, indent=2)},
            ],
        },
    ]

    return example


# Set random seed for reproducibility
random.seed(42)

In [ ]:
# Create multiple variations by applying the format function multiple times with different random questions
NUM_VARIATIONS = 3  # Create 3 question variations per example

all_train_variations = []
for i in range(NUM_VARIATIONS):
    random.seed(42 + i)  # Different seed for each variation
    varied_train = data["train"].map(format_webclick_with_varied_questions)
    all_train_variations.append(varied_train)

# Combine all variations
data["train"] = concatenate_datasets(all_train_variations)

# Format validation set once (no variations needed for validation)
random.seed(42)
data["validation"] = data["validation"].map(format_webclick_with_varied_questions)

print(f"Expanded training set: {len(data['train'])} examples ({len(data['train']) // NUM_VARIATIONS} original × {NUM_VARIATIONS} variations)")
print(f"Validation set: {len(data['validation'])} examples")
print("\nExample formatted data:")
data["train"][0]

Map:   0%|          | 0/1311 [00:00<?, ? examples/s]

Map:   0%|          | 0/328 [00:00<?, ? examples/s]

Expanded training set: 3933 examples (1311 original × 3 variations)
Validation set: 328 examples

Example formatted data:


{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1096x824>,
 'instruction': 'Click 8.45 heated terrace',
 'bbox': [0.47292712135036497,
  0.6317885315533981,
  0.6852189781021898,
  0.7004702669902912],
 'bucket': 'calendars',
 'messages': [{'content': [{'text': "You are a web automation assistant. Analyze the webpage screenshot and respond with valid JSON containing 'instruction', 'bounding_box' with x1/y1/x2/y2 coordinates, and 'category'.",
     'type': 'text'}],
   'role': 'system'},
  {'content': [{'text': None, 'type': 'image'},
    {'text': 'Where should I click here?', 'type': 'text'}],
   'role': 'user'},
  {'content': [{'text': '{\n  "instruction": "Click 8.45 heated terrace",\n  "bounding_box": {\n    "x1": 47,\n    "y1": 63,\n    "x2": 68,\n    "y2": 70\n  },\n  "category": "calendars"\n}',
     'type': 'text'}],
   'role': 'assistant'}]}

In [ ]:
model_id = "google/medgemma-1.5-4b-it"

In [ ]:
model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=model_kwargs["torch_dtype"],
    bnb_4bit_quant_storage=model_kwargs["torch_dtype"],
)

model = AutoModelForImageTextToText.from_pretrained(model_id, **model_kwargs)
processor = AutoProcessor.from_pretrained(model_id)
processor.tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/2.55k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

...

The image processor of type `Gemma3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
def collate_fn(examples: list[dict[str, Any]]):
    texts = []
    images = []
    for example in examples:
        images.append([example["image"].convert("RGB")])
        texts.append(processor.apply_chat_template(
            example["messages"], add_generation_prompt=False, tokenize=False
        ).strip())

    # Tokenize the texts and process the images
    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)

    # The labels are the input_ids, with the padding and image tokens masked in
    # the loss computation
    labels = batch["input_ids"].clone()

    # Mask image tokens
    image_token_id = [
        processor.tokenizer.convert_tokens_to_ids(
            processor.tokenizer.special_tokens_map["boi_token"]
        )
    ]
    # Mask tokens that are not used in the loss computation
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    labels[labels == 262144] = -100

    batch["labels"] = labels
    return batch

In [ ]:
num_train_epochs = 5
learning_rate = 1e-6

In [ ]:
args = SFTConfig(
    output_dir="medgemma-4b-it-sft-lora-webclick-json",
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="steps",
    eval_strategy="steps",
    eval_steps=10,
    learning_rate=learning_rate,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    push_to_hub=False,
    report_to="tensorboard",
    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_kwargs={"skip_prepare_dataset": True},
    remove_unused_columns = False,
    label_names=["labels"],
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False
)

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=data["train"],
    eval_dataset=data["validation"].shuffle().select(range(200)),  # Use subset of validation set for faster run
    peft_config=peft_config,
    processing_class=processor,
    data_collator=collate_fn,
)

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.


Step,Training Loss,Validation Loss
10,3.151974,3.164368
20,3.155552,3.160869
30,3.182512,3.162469


Done training now to check train

In [ ]:
trainer.save_model()

In [ ]:
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
!wget -nc -q "https://zenodo.org/records/1214456/files/CRC-VAL-HE-7K.zip"
!unzip -q CRC-VAL-HE-7K.zip

In [ ]:
def format_test_data(example: dict[str, Any]) -> dict[str, Any]:
    example["messages"] = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                },
                {
                    "type": "text",
                    "text": PROMPT,
                },
            ],
        },
    ]
    return example


test_data = load_dataset("./CRC-VAL-HE-7K", split="train")
test_data = test_data.shuffle(seed=42).select(range(1000))
test_data = test_data.map(format_test_data)

In [ ]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
REFERENCES = test_data["label"]


def compute_metrics(predictions: list[int]) -> dict[str, float]:
    metrics = {}
    metrics.update(accuracy_metric.compute(
        predictions=predictions,
        references=REFERENCES,
    ))
    metrics.update(f1_metric.compute(
        predictions=predictions,
        references=REFERENCES,
        average="weighted",
    ))
    return metrics

In [ ]:
test_data = test_data.cast_column(
    "label",
    ClassLabel(names=TISSUE_CLASSES)
)

LABEL_FEATURE = test_data.features["label"]

ALT_LABELS = dict([
    (label, f"({label.replace(': ', ') ')}") for label in TISSUE_CLASSES
])


def postprocess(prediction: list[dict[str, str]], do_full_match: bool=False) -> int:
    response_text = prediction[0]["generated_text"]
    if do_full_match:
        return LABEL_FEATURE.str2int(response_text)
    for label in TISSUE_CLASSES:
        # Search for `X: tissue type` or `(X) tissue type` in the response
        if label in response_text or ALT_LABELS[label] in response_text:
            return LABEL_FEATURE.str2int(label)
    return -1

First load a baseline preview

In [ ]:
pt_pipe = pipeline(
    "image-text-to-text",
    model=model_id,
    torch_dtype=torch.bfloat16,
)

# Set `do_sample = False` for deterministic responses
pt_pipe.model.generation_config.do_sample = False
pt_pipe.model.generation_config.pad_token_id = processor.tokenizer.eos_token_id

In [ ]:
pt_outputs = pt_pipe(
    text=test_data["messages"],
    images=test_data["image"],
    max_new_tokens=40,
    batch_size=64,
    return_full_text=False,
)

pt_predictions = [postprocess(out) for out in pt_outputs]

In [ ]:
pt_metrics = compute_metrics(pt_predictions)
print(f"Baseline metrics: {pt_metrics}")

Then load finetuned model

In [ ]:
ft_pipe = pipeline(
    "image-text-to-text",
    model=args.output_dir,
    processor=processor,
    torch_dtype=torch.bfloat16,
)

ft_pipe.model.generation_config.do_sample = False # Set `do_sample = False` for deterministic responses
ft_pipe.model.generation_config.pad_token_id = processor.tokenizer.eos_token_id
# Use left padding during inference idk why just do it
processor.tokenizer.padding_side = "left"

In [ ]:
ft_outputs = ft_pipe(
    text=test_data["messages"],
    images=test_data["image"],
    max_new_tokens=20,
    batch_size=64,
    return_full_text=False,
)

ft_predictions = [postprocess(out, do_full_match=True) for out in ft_outputs]

In [ ]:
ft_metrics = compute_metrics(ft_predictions)
print(f"Fine-tuned metrics: {ft_metrics}")